In [2]:
# read and inspect the data file
with open('./input.txt', 'r') as file:
    text = file.read()

print(f'the lenth of the file in characters is {len(text)}')

the lenth of the file in characters is 1115394


In [3]:
# take a look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
# get all the unique characters in the file
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(f'the number of unique characters in the file is {vocab_size}')


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
the number of unique characters in the file is 65


In [5]:
"""tokenizer the input texts"""

# create the map func from chars to integers
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda x: [stoi[ch] for ch in x]
decode = lambda x: ''.join([itos[i] for i in x])

# test the encode and decode functions
print(encode('hello world\n'))
print(decode(encode('hello world\n')))
# 不难发现, ' '是1, '\n'是0

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42, 0]
hello world



In [6]:
import torch
data = torch.tensor(encode(text), dtype=torch.long) # long = int64
print(data.shape, data.dtype)
print(data[:1000])

e:\apps\anaconda\envs\dl\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [7]:
"""Seperate the train and validation data"""
n = int(0.9 * len(data))
train_data, valid_data = data[:n], data[n:]

In [8]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [10]:
torch.manual_seed(43)
batch_size = 4 # the sequences we process in parallel
block_size = 8 # the maximum context length for predictions

def get_batch(split):
    # generate a small random batch data of inputs x and targets y
    data = train_data if split == 'train' else valid_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # starting index of each sequence
    x = torch.stack([data[i:i+block_size] for i in ix]) # input
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) # target
    return x, y

xb, yb = get_batch('train')
print(f"inputs:")
print(xb.shape)
print(xb)
print(f"targets:")
print(yb.shape)
print(yb)

print('-'*10)

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context} the target is {target}")

inputs:
torch.Size([4, 8])
tensor([[43,  1, 58, 53,  1, 51, 43,  6],
        [ 1, 57, 46, 43,  1, 61, 39, 57],
        [39, 42, 43,  1, 46, 47, 51,  1],
        [53, 59, 56,  1, 45, 56, 39, 41]])
targets:
torch.Size([4, 8])
tensor([[ 1, 58, 53,  1, 51, 43,  6,  1],
        [57, 46, 43,  1, 61, 39, 57,  1],
        [42, 43,  1, 46, 47, 51,  1, 58],
        [59, 56,  1, 45, 56, 39, 41, 43]])
----------
when input is tensor([43]) the target is 1
when input is tensor([43,  1]) the target is 58
when input is tensor([43,  1, 58]) the target is 53
when input is tensor([43,  1, 58, 53]) the target is 1
when input is tensor([43,  1, 58, 53,  1]) the target is 51
when input is tensor([43,  1, 58, 53,  1, 51]) the target is 43
when input is tensor([43,  1, 58, 53,  1, 51, 43]) the target is 6
when input is tensor([43,  1, 58, 53,  1, 51, 43,  6]) the target is 1
when input is tensor([1]) the target is 57
when input is tensor([ 1, 57]) the target is 46
when input is tensor([ 1, 57, 46]) the target

In [11]:
""" Starting from basic bi-gram language model """
from regex import B
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(42)

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        # idx and targets are both of shape (B, T)
        logits = self.token_embedding_table(idx) # (B, T, C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) #! Channel as second dimension for cross-entropy of pytorch
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits,loss
    
    def generate(self, idx, max_new_tokens):
        """generat from (B, T) to (B, T+max_new_tokens)"""
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            logits, loss = self(idx) # calling forward to get the logits
            logits = logits[:, -1, :] # becomes (B, C)
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx


m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape, loss)

idx = torch.zeros((1, 1), dtype=torch.long)
print(decode (m.generate(idx, max_new_tokens=100)[0].tolist()) )

torch.Size([32, 65]) tensor(4.6329, grad_fn=<NllLossBackward0>)

o$,q&IWqW&xtCjaB?ij&bYRGkF?b; f ,CbwhtERCIfuWr,DzJERjhLlVaF&EjffPHDFcNoGIG'&$qXisWTkJPw
 ,b Xgx?D3sj


In [12]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [13]:
batch_size = 32
for steps in range(10000):
    
    # sample a batch of data
    xb, yb = get_batch('train')
    
    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())


2.5233194828033447


In [14]:
print(decode (m.generate(idx, max_new_tokens=100)[0].tolist()) )



QUDUThe chas.
F lisen tabr:
LI mus nk,
A: al l ayo cenghe's therinvar,
TEsen ithawaneit at islinerai


### The mathematical traick in self-attention mechanism

In [15]:
# consider the example
torch.manual_seed(42)
B, T, C = 4,8,2
x = torch.randn(B, T, C)

# version 1: using a for loop
# bow: bag of words
xbow = torch.zeros((B, T, C))
# *仅仅对t之前的所有词做一个语义平均, 但是这里并没有使用矩阵乘法来优化
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1, :] # (t, C)
        xbow[b, t] = xprev.mean(dim=0)
print(x[0])

tensor([[ 1.9269,  1.4873],
        [ 0.9007, -2.1055],
        [ 0.6784, -1.2345],
        [-0.0431, -1.6047],
        [-0.7521,  1.6487],
        [-0.3925, -1.4036],
        [-0.7279, -0.5594],
        [-0.7688,  0.7624]])


In [16]:
xbow[0]

tensor([[ 1.9269,  1.4873],
        [ 1.4138, -0.3091],
        [ 1.1687, -0.6176],
        [ 0.8657, -0.8644],
        [ 0.5422, -0.3617],
        [ 0.3864, -0.5354],
        [ 0.2272, -0.5388],
        [ 0.1027, -0.3762]])

In [17]:
"""Using matrix multiplication to compute the bag of words representation"""
# version 2: using matrix multiplication
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(dim=1, keepdim=True)

# when pytorch find that the shape is not match, it will automatically broadcast the tensor
# here when wei is (T, T) and x is (B, T, C), it will automatically broadcast wei to (B, T, T)
# every batch element would do (T, T) @ (T, C) => (T, C) in parallel
xbow2 = wei @ x # (T,T) @ (B, T, C) => (B, T, T)) @ (B, T, C) => (B, T, C)
torch.allclose(xbow, xbow2) # check if the two tensors are close

True

In [18]:
# version 3: using Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [19]:
# version 4: self-attention

torch.manual_seed(42)
B, T, C = 4, 8, 32 # batch, time, channels
x = torch.randn(B, T, C)

# single head self-attention
head_size = 16
key = nn.Linear(C, head_size, bias = False)
query = nn.Linear(C, head_size, bias = False)
value = nn.Linear(C, head_size, bias = False)
k = key(x) # (B, T, H)
q = query(x) # (B, T, H)
wei = q @ k.transpose(-2, -1) / head_size**0.5 # (B, T, H) @ (B, H, T) => (B, T, T)

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
v = value(x) # (B, T, H)
out = wei @ v # (B, T, T) @ (B, T, H) => (B, T, H)


out.shape

torch.Size([4, 8, 16])

Note
- we would see the attention process as a communication method between the query and the key-value pairs.
- there is no notion of space, this is why we need to positionally encode the input sequence.
- each sample across batch dimension is of course processed completely independently and never 'talk' to each other.
- In an 'encoder' attention block just delete the tril allowing tokens to communicate with each other in the future.While in an 'decoder' block we shall not do this, because we want to prevent the tokens from the future to communicate with the tokens from the past.
- 'self-attention' means the keys and values are produced from the same soruce as the queries.In 'cross attention' the keys and values are produced from a different source.While the queries are still produced from x.
- ''Scaled'' attention divides the wei by 1/sqrt(head_size), this makes the input Q,K are unit variance, wei will be unit variance as well, this is important for the softmax function to work properly.
- [对于可能过于极端的初始化值(注意我们是随机初始化的), 可能导致整个向量softmax过后趋向于0或者1(独热), 这样会导致梯度消失或者梯度爆炸, 为了避免这种情况, 我们需要对输入进行正则化, 使其均值为0, 方差为1, 这样softmax函数才能正常工作]

In [20]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1) # * head_size**-0.5

In [21]:
k.var(), q.var(), wei.var()

(tensor(1.0392), tensor(0.9791), tensor(12.4662))

We found here we have unit gaussion input for k and q, while the output wei isn't unit gaussion

In [22]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1)  * head_size**-0.5

In [23]:
k.var(), q.var(), wei.var()

(tensor(1.0346), tensor(0.9618), tensor(0.9446))

In [24]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim = -1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [25]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim = -1)


tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

here we would see sharper values

In [32]:
class BatchNorm1d:
    
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.momentum = momentum
        self.training = True
        # parameters trained with backprop
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        # buffers
        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim)
    
    def __call__(self, x) :
        if self.training:
            xmean = x.mean(0, keepdim=True) # mean along the batch dimension
            xvar = x.var(0, keepdim=True) # variance along the batch dimension
        else: 
            xmean = self.running_mean
            xvar = self.running_var
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize
        self.out = self.gamma * xhat + self.beta
        # update the buffers
        if self.training:
            with torch.no_grad():
                self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
                self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]

torch.manual_seed(42)
module = BatchNorm1d(100)
x = torch.randn(32, 100) # batch, features
x = module(x)
print(x.mean(), x.var(), x.shape)

tensor(8.9407e-10) tensor(0.9690) torch.Size([32, 100])


In [33]:
x[:, 0].mean(), x[:, 0].std()

(tensor(1.4901e-08), tensor(1.0000))

In [35]:
x[0, :].mean(), x[0, :].std()

(tensor(0.0082), tensor(0.9901))

the rows are not going to be normalized, but the columns are going to be normalized in batchnorm

In [39]:
class LayerNorm1d:
    
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        # parameters trained with backprop
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
    
    def __call__(self, x) :
        xmean = x.mean(1, keepdim=True) # mean along the batch dimension
        xvar = x.var(1, keepdim=True) # variance along the batch dimension
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize
        self.out = self.gamma * xhat + self.beta
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]

torch.manual_seed(42)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch, features
x = module(x)
print(x.mean(), x.var(), x.shape)

tensor(-1.7881e-09) tensor(0.9903) torch.Size([32, 100])


In [40]:
x[:, 0].mean(), x[:, 0].std()

(tensor(-0.1428), tensor(1.0629))

In [41]:
x[0, :].mean(), x[0, :].std()

(tensor(-2.6226e-08), tensor(1.0000))

We could see that the columns are normalized, but the rows are not normalized